In [4]:
import pandas as pd

df = pd.read_csv("customer_support_tickets.csv")
df.head()

,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaN,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaN,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0


In [5]:
df.shape

(8469, 17)

In [6]:
df.columns

Index(['Ticket ID', 'Customer Name', 'Customer Email', 'Customer Age',
       'Customer Gender', 'Product Purchased', 'Date of Purchase',
       'Ticket Type', 'Ticket Subject', 'Ticket Description', 'Ticket Status',
       'Resolution', 'Ticket Priority', 'Ticket Channel',
       'First Response Time', 'Time to Resolution',
       'Customer Satisfaction Rating'],
      dtype='object')

In [7]:
df.isnull().sum()

,0
Ticket ID,0
Customer Name,0
Customer Email,0
Customer Age,0
Customer Gender,0
Product Purchased,0
Date of Purchase,0
Ticket Type,0
Ticket Subject,0
Ticket Description,0


In [8]:
df["Ticket Description"].head(5)

,Ticket Description
0,I'm having an issue with the {product_purchase...
1,I'm having an issue with the {product_purchase...
2,I'm facing a problem with my {product_purchase...
3,I'm having an issue with the {product_purchase...
4,I'm having an issue with the {product_purchase...


In [9]:
import re

def clean_text(text):
    text = text.lower()
    text = text.replace("_", " ")
    text = re.sub(r'[{}]', '', text)
    text = re.sub(r'[^a-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

df["cleaned_text"] = df["Ticket Description"].apply(clean_text)

df[["Ticket Description", "cleaned_text"]].head(5)

,Ticket Description,cleaned_text
0,I'm having an issue with the {product_purchase...,im having an issue with the product purchased ...
1,I'm having an issue with the {product_purchase...,im having an issue with the product purchased ...
2,I'm facing a problem with my {product_purchase...,im facing a problem with my product purchased ...
3,I'm having an issue with the {product_purchase...,im having an issue with the product purchased ...
4,I'm having an issue with the {product_purchase...,im having an issue with the product purchased ...


In [10]:
def classify_issue(text):
    if "refund" in text or "money back" in text:
        return "REFUND"

    elif "payment" in text or "billing" in text or "charged" in text or "card" in text:
        return "PAYMENT"

    elif "login" in text or "password" in text or "otp" in text or "account" in text:
        return "LOGIN"

    elif "delivery" in text or "shipping" in text or "delayed" in text:
        return "DELIVERY"

    elif "not working" in text or "error" in text or "issue" in text or "problem" in text:
        return "BUG"

    else:
        return "GENERAL"


# Apply classification
df["issue_category"] = df["cleaned_text"].apply(classify_issue)

# Check result
df[["cleaned_text", "issue_category"]].head(10)


,cleaned_text,issue_category
0,im having an issue with the product purchased ...,PAYMENT
1,im having an issue with the product purchased ...,BUG
2,im facing a problem with my product purchased ...,BUG
3,im having an issue with the product purchased ...,BUG
4,im having an issue with the product purchased ...,DELIVERY
5,im facing a problem with my product purchased ...,BUG
6,im unable to access my product purchased accou...,LOGIN
7,im having an issue with the product purchased ...,BUG
8,im having an issue with the product purchased ...,BUG
9,my product purchased is making strange noises ...,BUG


In [11]:
def assign_priority(text):
    # P0 – Critical
    p0_keywords = [
        "urgent", "asap", "immediately", "critical",
        "failed", "failure", "not working", "system down",
        "service down", "crashed", "crash", "error",
        "blocked", "stuck", "frozen", "unresponsive",
        "cannot use", "unable to use", "stopped working",
        "service unavailable"
    ]

    # P1 – High
    p1_keywords = [
        "refund", "money back", "charged", "double charged",
        "overcharged", "payment", "billing", "invoice",
        "card", "credit card", "debit card", "transaction",
        "transaction failed", "payment failed",
        "amount deducted", "balance deducted",
        "unauthorized", "fraud",
        "account locked", "account suspended",
        "cannot login", "login failed",
        "password issue", "otp issue", "access denied"
    ]

    # P2 – Medium
    p2_keywords = [
        "delay", "delayed", "slow", "slowness", "lag",
        "delivery", "shipping", "shipment", "dispatch",
        "courier", "late", "pending", "processing",
        "taking time", "not yet received",
        "order status", "tracking issue",
        "response delay", "waiting"
    ]

    # Check priorities in order
    if any(word in text for word in p0_keywords):
        return "P0"
    elif any(word in text for word in p1_keywords):
        return "P1"
    elif any(word in text for word in p2_keywords):
        return "P2"
    else:
        return "P3"


# Apply priority assignment
df["priority_level"] = df["cleaned_text"].apply(assign_priority)

# Verify result
df[["cleaned_text", "issue_category", "priority_level"]].head(10)


,cleaned_text,issue_category,priority_level
0,im having an issue with the product purchased ...,PAYMENT,P1
1,im having an issue with the product purchased ...,BUG,P3
2,im facing a problem with my product purchased ...,BUG,P3
3,im having an issue with the product purchased ...,BUG,P3
4,im having an issue with the product purchased ...,DELIVERY,P2
5,im facing a problem with my product purchased ...,BUG,P3
6,im unable to access my product purchased accou...,LOGIN,P0
7,im having an issue with the product purchased ...,BUG,P3
8,im having an issue with the product purchased ...,BUG,P2
9,my product purchased is making strange noises ...,BUG,P3


In [12]:
from datetime import datetime, timedelta

# Function to assign SLA hours
def assign_sla(priority):
    if priority == "P0":
        return 2
    elif priority == "P1":
        return 6
    elif priority == "P2":
        return 24
    else:  # P3
        return 48

# Apply SLA hours
df["sla_hours"] = df["priority_level"].apply(assign_sla)

# Get current time
current_time = datetime.now()

# Calculate due time
df["due_time"] = df["sla_hours"].apply(
    lambda x: current_time + timedelta(hours=x)
)

# Check result
df[["priority_level", "sla_hours", "due_time"]].head(10)

,priority_level,sla_hours,due_time
0,P1,6,2025-12-26 16:57:35.714048
1,P3,48,2025-12-28 10:57:35.714048
2,P3,48,2025-12-28 10:57:35.714048
3,P3,48,2025-12-28 10:57:35.714048
4,P2,24,2025-12-27 10:57:35.714048
5,P3,48,2025-12-28 10:57:35.714048
6,P0,2,2025-12-26 12:57:35.714048
7,P3,48,2025-12-28 10:57:35.714048
8,P2,24,2025-12-27 10:57:35.714048
9,P3,48,2025-12-28 10:57:35.714048


In [13]:
# ===============================
# SUPPORT MANAGER SUMMARY REPORT
# ===============================

print("\n========== SUPPORT MANAGER SUMMARY REPORT ==========\n")

# 1️⃣ Overall Summary
total_tickets = len(df)
print(f"Total Tickets Received: {total_tickets}\n")


# 2️⃣ Issue Category Breakdown
issue_summary = df["issue_category"].value_counts().reset_index()
issue_summary.columns = ["Issue Category", "Number of Tickets"]

print("=== Issue Category Breakdown ===")
print(issue_summary)
print()


# 3️⃣ Priority Level Breakdown
priority_summary = df["priority_level"].value_counts().reset_index()
priority_summary.columns = ["Priority Level", "Number of Tickets"]

print("=== Priority Level Breakdown ===")
print(priority_summary)
print()


# 4️⃣ SLA Overview
sla_summary = df.groupby("priority_level")["sla_hours"].mean().reset_index()
sla_summary.columns = ["Priority Level", "Average SLA (Hours)"]

print("=== SLA Overview ===")
print(sla_summary)
print()


# 5️⃣ Urgent Tickets Snapshot (P0 & P1)
urgent_tickets = df[df["priority_level"].isin(["P0", "P1"])]

print("=== Urgent Tickets Snapshot (Top 10) ===")
print(
    urgent_tickets[
        ["issue_category", "priority_level", "sla_hours", "due_time"]
    ].head(10)
)
print()


# 6️⃣ Export Report Files (Optional but Professional)
issue_summary.to_csv("issue_summary_report.csv", index=False)
priority_summary.to_csv("priority_summary_report.csv", index=False)

print("Issue summary and priority summary reports exported successfully.")

print("\n========== END OF REPORT ==========\n")



========== SUPPORT MANAGER SUMMARY REPORT ==========

Total Tickets Received: 8469

=== Issue Category Breakdown ===
  Issue Category  Number of Tickets
0            BUG               7104
1          LOGIN                731
2        PAYMENT                215
3        GENERAL                169
4         REFUND                162
5       DELIVERY                 88

=== Priority Level Breakdown ===
  Priority Level  Number of Tickets
0             P3               5289
1             P0               1887
2             P2                924
3             P1                369

=== SLA Overview ===
  Priority Level  Average SLA (Hours)
0             P0                  2.0
1             P1                  6.0
2             P2                 24.0
3             P3                 48.0

=== Urgent Tickets Snapshot (Top 10) ===
   issue_category priority_level  sla_hours                   due_time
0         PAYMENT             P1          6 2025-12-26 16:57:35.714048
6           LOGIN   

In [14]:
# Export full final dataset
df.to_csv("final_ticket_output.csv", index=False)

print("Final ticket CSV created: final_ticket_output.csv")


Final ticket CSV created: final_ticket_output.csv
